In [1]:
import pandas as pd
import numpy as np

In [2]:
#定义数据结构
class person:
    age = 0
    gender = 0
    id = 0
    info_list = []
    score = 0
    analysis = None

In [3]:
#初始化
person_info = []

In [4]:
#读数据
data = pd.read_csv('data_project.csv')
data

,age,createdAt,gender,id,list,objectId,score,updatedAt
0,12,2022-12-01T11:31:05.032Z,1,5,"[[0,4,""Small"",0,527,""wrong""],[0,3,""Big"",3,27,""...",6388907994337f1978cf1e27,9,2022-12-01T11:31:05.032Z


In [5]:
#数据存入person_info
for index,row in data.iterrows():
    new_person = person
    new_person.age = row['age']
    new_person.gender = row['gender']
    new_person.id = row['id']
    new_person.score = row['score']
    
    info = row['list'].split(']')[:-2]
    new_list = []
    for i in info:
        new_list.append(i.split('[')[-1:][0].split(','))
        
    new_person.info_list = new_list
    new_person.analysis = pd.DataFrame(new_person.info_list,columns = ['small_num', 'big_num', 'type', 'answer', 'time_cost', 'correctness'])
    diff_list = [0]
    for i in range(1,len(new_list)):
        diff_list.append(int(new_list[i][4]) - int(new_list[i-1][4]))
    #diff是指与上一次的时间差
    new_person.analysis['diff'] = diff_list
    count = 0
    current_type = new_list[0][2]
    if_change_list = [0]
    for i in new_list[1:]:
        if i[2] == current_type:
            count += 1
            if_change_list.append(100*count)
        else:
            count = 0
            current_type = i[2]
            if_change_list.append(-100)
    #change是指需要忽略的类型是否改变，正数表示未变，负数表示变了，且未变的时间越长数值越大
    new_person.analysis['change'] = if_change_list
    
    person_info.append(new_person)

In [6]:
#所有人的数据的总和 all_gender_data
all_gender_data = pd.DataFrame([],columns = ['small_num', 'big_num', 'type', 'answer', 'time_cost', 'correctness', 'diff', 'change'])
for i in person_info:
    all_gender_data = all_gender_data.append(i.analysis)
    
#所有男性的数据总和 all_male_data
all_male_data = pd.DataFrame([],columns = ['small_num', 'big_num', 'type', 'answer', 'time_cost', 'correctness', 'diff', 'change'])
for i in person_info:
    if i.gender == 1:
        all_male_data = all_male_data.append(i.analysis)

#所有女性的数据总和 all_female_data
all_female_data = pd.DataFrame([],columns = ['small_num', 'big_num', 'type', 'answer', 'time_cost', 'correctness', 'diff', 'change'])
for i in person_info:
    if i.gender == 0:
        all_female_data = all_female_data.append(i.analysis)       
        
all_gender_data

,small_num,big_num,type,answer,time_cost,correctness,diff,change
0,0,4,"""Small""",0,527,"""wrong""",0,0
1,0,3,"""Big""",3,27,"""wrong""",-500,-100
2,1,4,"""Big""",4,18,"""wrong""",-9,100
3,3,1,"""Big""",1,60,"""wrong""",42,200
4,1,2,"""Small""",1,8,"""wrong""",-52,-100
...,...,...,...,...,...,...,...,...
56,2,4,"""Small""",2,27,"""wrong""",-49,-100
57,4,1,"""Big""",1,1,"""wrong""",-26,-100
58,2,1,"""Big""",1,2,"""wrong""",1,100
59,4,2,"""Small""",4,7,"""wrong""",5,-100


In [8]:
from sklearn.datasets import fetch_openml
import plotly
pd.options.plotting.backend = "plotly" # 设置pandas调用plotly绘图
all_gender_data.plot(y = [all_gender_data['diff'], all_gender_data['change']], kind = 'line')